# 第一步 环境和数据准备


In [ ]:
#克隆zero_nlp项目
!git clone https://github.com/yuanzhoulvpi2017/zero_nlp
# 把项目复制到colab对应文件夹
!cp -r /content/zero_nlp/simple_thu_chatglm6b/* /content/
# 安装环境
!pip install protobuf==3.20.0 transformers icetk cpm_kernels peft datasets

注意：在上一步结束后，需要检查一下在nodebook的同级文件夹下是否有thuglm这个simple_thu_chatglm6b项目中的文件夹！


In [ ]:
# 下载模型权重
import os
import requests

# 在 /content 目录下创建一个文件夹
download_folder = "/content/thuglm"
os.makedirs(download_folder, exist_ok=True)

# 列出要下载的文件路径
files_to_download = [
    "config.json",
    "pytorch_model-00001-of-00008.bin",
    "pytorch_model-00002-of-00008.bin",
    "pytorch_model-00003-of-00008.bin",
    "pytorch_model-00004-of-00008.bin",
    "pytorch_model-00005-of-00008.bin",
    "pytorch_model-00006-of-00008.bin",
    "pytorch_model-00007-of-00008.bin",
    "pytorch_model-00008-of-00008.bin",
    "ice_text.model",
]

# 下载文件
base_url = "https://huggingface.co/THUDM/chatglm-6b/resolve/main/"

for file_path in files_to_download:
    print(f"Downloading {file_path}...")
    response = requests.get(base_url + file_path)
    response.raise_for_status()  # 如果发生错误，抛出异常

    # 将文件内容写入本地文件
    with open(os.path.join(download_folder, file_path), "wb") as f:
        f.write(response.content)

print("All files downloaded successfully.")


# 第二步 数据集转换，把alpaca类型数据集转成chatglm的形式

我们感觉可能是数据出问题了，但是不知道哪里出了问题。。。

In [ ]:
# 把数据集转化一下，变成chatglm的格式

import json

# 读取json文件，这里的json得自己合一下我们的训练数据
with open('/content/1.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 将json格式转换为数组对话格式
dialogue_data = []
for i, d in enumerate(data):
    dialogue_data.append('[Round {}]\n问：{}'.format((1 // 2) + 1, d['instruction']) + '\n答：{}\n'.format(d['output']))

# 打印结果
print(dialogue_data)

import csv

with open('/content/googdata.csv', mode='w',  encoding='utf-8',newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['content'])
    for item in dialogue_data:
        writer.writerow([item])

In [3]:
# MyTrainerz在colab上运行的时候164行左右会出问题
from MyTrainer import Trainer


from transformers import TrainingArguments
from transformers import DataCollatorForLanguageModeling
import random
from glob import glob
from datasets import load_dataset, DatasetDict # 加载数据用的
from transformers import AutoTokenizer, AutoModel

# lora已经在peft里面实现了，因此使用peft包即可
from peft import get_peft_model, LoraConfig, TaskType

ImportError: cannot import name 'can_return_loss' from 'transformers.utils' (/home/user/anaconda3/envs/codegeexzy/lib/python3.9/site-packages/transformers/utils/__init__.py)

In [ ]:
# 加载模型和权重
tokenizer = AutoTokenizer.from_pretrained("thuglm", trust_remote_code=True)
model = AutoModel.from_pretrained(
    "thuglm", trust_remote_code=True).half().cuda()

# 第三步 使用lora
我们这里训练了很多参数，loss也降低了，但是不知道为啥模型还是没反应

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1,
    # ['dense','dense_h_to_4h','dense_4h_to_h'] # 'query_key_value',
    target_modules=['query_key_value', ],
)
model = get_peft_model(model, peft_config)

In [ ]:
# 加载训练和测试数据
import random

random.seed(42)

train_file = "/content/googdata.csv"
test_file = "/content/googdata_tes.csv"

raw_datasets = load_dataset("csv", data_files={
                            'train': train_file, 'valid': test_file}, cache_dir="cache_data")


# 数据处理，我感觉这边会不会有问题..

In [ ]:
context_length = 512 # 这个大小，基本不影响显存，因此设置为1024也行，目前不知道chatglm要求的文本长度上限为多少

def tokenize(element):
    outputs = tokenizer(
        element["content"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length <= context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["train"].column_names
)
tokenized_datasets

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


# 训练
loss降低了，val也挺好，但是就是模型没动静。。

In [ ]:
args = TrainingArguments(
    output_dir="test003",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_steps=100,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=100,
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
)
trainer.train()

# 测试

In [ ]:
from transformers import AutoTokenizer
from thuglm.modeling_chatglm import ChatGLMForConditionalGeneration
import torch

In [ ]:
# 这个是我们训练好的模型
model = ChatGLMForConditionalGeneration.from_pretrained("test003/checkpoint-200").cuda() #
tokenizer = AutoTokenizer.from_pretrained("thuglm", trust_remote_code=True)


In [ ]:
with torch.autocast("cuda"):
    res, history = model.chat(tokenizer=tokenizer, query="今天是星期几？")
        # res = model.forward(input_ids=all_input.get('input_ids').cuda())
    print(res)